In [1]:
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [7]:
data = pd.read_csv("abcnews-date-text.csv")

In [8]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [14]:
data = data[0:(data.shape[0]/4)]
data.shape

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.range.RangeIndex'> with these indexers [275915.75] of <class 'float'>

In [93]:
train = data['headline_text']
test = data.drop('headline_text', axis = 1)

In [94]:
len(test)

1000

In [95]:
len(train)

1000

In [100]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train))

In [101]:
trainvect = CountVectorizer()

In [102]:
Trainfeature = trainvect.fit_transform(trainheadlines)

In [103]:
####Detailed view of Document Count Matrix
DTM_With_Colm = pd.DataFrame(Trainfeature.toarray(),columns= trainvect.get_feature_names())

In [104]:
Trainfeature.shape

(1000, 2743)

Model Logistic Regression

In [105]:
Logis = LogisticRegression()

In [108]:
train.head()

0    aba decides against community broadcasting lic...
1       act fire witnesses must be aware of defamation
2       a g calls for infrastructure protection summit
3             air nz staff in aust strike for pay rise
4        air nz strike to affect australian travellers
Name: headline_text, dtype: object

In [109]:
Model1 = Logis.fit(Trainfeature,train)

/home/shivi/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/shivi/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [111]:
Model1

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [113]:
testheadlines =[]
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test))

In [114]:
len(testheadlines)

1000

In [115]:
Testfeature = trainvect.transform(testheadlines)

In [116]:
Testfeature.shape

(1000, 2743)

In [117]:
Predicted = Model1.predict(Testfeature)

In [118]:
Predicted.shape

(1000,)

Model Naive Bayes

In [122]:
from sklearn.naive_bayes import MultinomialNB

Nb = MultinomialNB()

In [123]:
Model2 = Nb.fit(Trainfeature,train)

In [124]:
Nbpredicted = Model2.predict(Testfeature)

In [125]:
Nbpredicted.shape

(1000,)

In [126]:

import numpy as np
from sklearn.metrics import accuracy_score
y_NaviBayes = Nbpredicted
y_true = test
accuracy_score(y_NaviBayes,y_true)

/home/shivi/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:182: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


0.0

In [127]:
x_Logist = Predicted
x_true = test
accuracy_score(x_Logist,x_true)


0.0

**Ngram Model**

In [128]:
advvect = CountVectorizer(ngram_range=(1,2))
%time
advancedtrain = advvect.fit_transform(trainheadlines)
advancedtrain.shape


CPU times: user 11 µs, sys: 1 µs, total: 12 µs
Wall time: 33.1 µs


(1000, 8663)

Model Naive Bayes - Ngram

In [129]:
advmodel = MultinomialNB()
advancemodel = advmodel.fit(advancedtrain,train)
advancetest = advvect.transform(testheadlines)
advNBprediction = advmodel.predict(advancetest) 
advNBprediction.shape




(1000,)

In [32]:
pd.crosstab(test["Label"],advNBprediction, rownames=["Acutal"],colnames=["Predicted"])

Predicted,0,1
Acutal,,
0,3,183
1,0,192


In [130]:
x_adNB = advNBprediction
x_test = test

In [131]:
accuracy_score(x_test,x_adNB)

/home/shivi/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:182: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


0.0

In [132]:
from sklearn import metrics
metrics.accuracy_score(x_test,x_adNB)

0.0

**Latent Dirichlet Allocation**
Topic Modeling 


In [133]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

In [ ]:
%time
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
#Our Document
trainheadlines

# list for tokenized documents in loop
texts = []

# loop through document list
for i in trainheadlines:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 6.91 µs


In [ ]:
%time
#generate LDA
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=1,chunksize=10000,update_every=1)